<h1>CNN Model</h1>

In [1]:
from pycocotools.coco import COCO
import matplotlib
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as functions
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
import re

In [2]:
DATADIR = "cocodoom/"

dataSplit, run = "run-full-train", "run1"

annFile = '{}{}.json'.format(DATADIR,dataSplit)

In [3]:
coco = COCO(annFile)

loading annotations into memory...
Done (t=14.31s)
creating index...
index created!


In [17]:
player_positions = {}

player_regex = re.compile(r"^(?P<tic>\d+)\s+player:(?P<x>-?\d+\.\d+),(?P<y>-?\d+\.\d+),(?P<z>-?\d+\.\d+),(?P<angle>-?\d+\.\d+)$")

with open(DATADIR+run+"/log.txt", 'r') as log_file:
    for line in log_file:
        if "player" in line:
            line = line.strip()
            tic, stats = line.split("player:")
            x, y, z, angle = stats.split(",")

            # Store position in the dictionary
            player_positions[int(tic)] = (float(x), float(y), float(z), float(angle))

In [9]:
class DoomMotionDataset(Dataset):
    def __init__(self, transform=None):
        self.img_ids = coco.getImgIds()
        self.transform = transform

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        # Load the RGB image
        rgb_filename = coco.loadImgs(self.img_ids[idx])[0]['file_name']
        tic = int(rgb_filename.replace(".png", "").split("/")[-1])
        next_tic = tic+1
        previous_tic = tic-1

        player_position = player_positions[tic]
        
        if next_tic not in player_positions:
            next_player_position = player_position
        else:
            next_player_position = player_positions[next_tic]
        dx = next_player_position[0] - player_position[0]
        dy = next_player_position[1] - player_position[1]
        dz = next_player_position[2] - player_position[2]
        dangle = next_player_position[3] - player_position[3]
        dangle = (dangle + np.pi) % 2*np.pi - np.pi
        next_motion_vector = (dx, dy, dz, dangle)

        if previous_tic not in player_positions:
            prev_player_position = player_position
        else :
            prev_player_position = player_positions[previous_tic]
        dx = next_player_position[0] - player_position[0]
        dy = next_player_position[1] - player_position[1]
        dz = next_player_position[2] - player_position[2]
        dangle = next_player_position[3] - player_position[3]
        dangle = (dangle + np.pi) % 2*np.pi - np.pi
        prev_motion_vector = (dx, dy, dz, dangle)
            
        prev_motion_vector = torch.tensor(prev_motion_vector, dtype=torch.float32)
        next_motion_vector = torch.tensor(next_motion_vector, dtype=torch.float32)
        
        return prev_motion_vector, next_motion_vector


In [10]:
class NeuralNetwork(nn.Module):
  def __init__(self, activation_function=functions.relu, device=torch.device("cpu")):
    super(NeuralNetwork, self).__init__()

    self.activation_function = activation_function
    self.fc1 = nn.Linear(4, 64).to(device)
    self.bn1 = nn.BatchNorm1d(num_features=64).to(device)

    self.fc2 = nn.Linear(64, 128).to(device)
    self.bn2 = nn.BatchNorm1d(num_features=128).to(device)

    self.fc3 = nn.Linear(128, 64).to(device)
    self.bn3 = nn.BatchNorm1d(num_features=64).to(device)

    self.out = nn.Linear(64, 4).to(device)

  def forward(self, x):
    x = self.activation_function(self.bn1(self.fc1(x)))
    x = self.activation_function(self.bn2(self.fc2(x)))
    x = self.activation_function(self.bn3(self.fc3(x)))
    x = self.out(x)
    return x

In [12]:
batch_size = 256
learning_rate = 1e-3
num_epochs = 300
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralNetwork(device=device).to(device)

train_dataset = DoomMotionDataset()
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

# Loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
    for batch_idx, (inputs, targets) in enumerate(progress_bar):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
    
        loss = criterion(outputs, targets)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        progress_bar.set_postfix({
            "batch_loss": loss.item(),
            "batch_index": batch_idx + 1,
            "batch_size": inputs.size(0)
        })

    # Average loss per epoch
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), "doom_motion_model.pth")

cuda



poch 1/300: 100%|██████████| 991/991 [00:09<00:00, 102.47batch/s, batch_loss=16.1, batch_index=991, batch_size=220]

Epoch 1, Loss: 436.0378



poch 2/300: 100%|██████████| 991/991 [00:09<00:00, 102.78batch/s, batch_loss=14.5, batch_index=991, batch_size=220]

Epoch 2, Loss: 405.7944


...

Epoch 298, Loss: 4.6331



poch 299/300: 100%|█████████| 991/991 [00:09<00:00, 99.96batch/s, batch_loss=6.63, batch_index=991, batch_size=220]

Epoch 299, Loss: 5.2045


Epoch 300/300: 100%|█████████| 991/991 [00:11<00:00, 90.00batch/s, batch_loss=6.54, batch_index=991, batch_size=220]

Epoch 300, Loss: 5.5278


In [18]:
dataSplit, run = "run-full-test", "run3"

annFile = '{}{}.json'.format(DATADIR,dataSplit)

In [19]:
coco = COCO(annFile)

loading annotations into memory...
Done (t=6.31s)
creating index...
index created!


In [20]:
model.eval()  # Set the model to evaluation mode
running_loss = 0.0

test_dataset = DoomMotionDataset()
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
progress_bar = tqdm(test_loader, desc="Testing", unit="batch")

with torch.no_grad():  # Disable gradient calculations for evaluation
    for batch_idx, (inputs, targets) in enumerate(progress_bar):
        inputs, targets = inputs.to(device), targets.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        running_loss += loss.item()
        
        progress_bar.set_postfix({
            "batch_loss": loss.item(),
            "batch_index": batch_idx + 1,
            "batch_size": inputs.size(0)
        })

# Average loss over all batches
test_loss = running_loss / len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

Testing: 100%|█████████████| 462/462 [00:03<00:00, 134.77batch/s, batch_loss=0.762, batch_index=462, batch_size=122]

Test Loss: 174.9421
